# About this notebook

This notebook integrates previously acquired gridMet data for San Diego county with historical fire perimeters. It creates two new features for the gridMet dataframe that indicate (1) whether a fire occurred at the location and date specified by the record and (2) the acres burned by the fire if so. 

In [1]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
%%time
df = pd.read_parquet('gridMet.parquet.gz').reset_index()
df = df[~(df.date.isna() | df.precipitation_amount_mm.isna())]
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
df.drop(['latitude', 'longitude'], axis=1, inplace=True)

Wall time: 59 s


In [3]:
%%time
# Expand the latitude and longitude from a point to a rectangle.
# The extent of the rectangle is 1/24th of a degree in both directions,
# which is the displacement between points of the original gridMet data.
from shapely.geometry import Polygon
dx, dy = 1/24, 1/24
def to_box(p):
    poly = Polygon([(p.x,      p.y),
                    (p.x + dx, p.y),
                    (p.x + dx, p.y + dy),
                    (p.x,      p.y + dy),
                   ])
    return poly
df = df.set_geometry(df.geometry.apply(to_box))
df.head()

Wall time: 8min 42s


,date,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa,geometry
0,2000-01-01,0.0,40.5,0.00600,139.7,222.0,2.1,292.0,282.2,31.0,12.3,12.1,48.0,1.8,0.69,"POLYGON ((-117.97500 33.56667, -117.93333 33.5..."
1,2000-01-01,0.0,40.9,0.00593,136.6,222.0,2.1,291.9,282.2,32.0,12.2,12.0,48.0,1.8,0.70,"POLYGON ((-117.93333 33.56667, -117.89167 33.5..."
2,2000-01-01,0.0,41.2,0.00580,133.2,222.0,2.3,292.1,282.1,33.0,11.8,11.5,50.0,1.9,0.72,"POLYGON ((-117.89167 33.56667, -117.85000 33.5..."
3,2000-01-01,0.0,40.8,0.00577,129.8,218.0,2.4,292.0,281.4,34.0,11.8,11.4,50.0,1.9,0.69,"POLYGON ((-117.85000 33.56667, -117.80833 33.5..."
4,2000-01-01,0.0,38.2,0.00547,126.5,218.0,2.5,292.6,281.0,37.0,10.7,10.5,55.0,2.1,0.78,"POLYGON ((-117.80833 33.56667, -117.76667 33.5..."


In [4]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 6501450 entries, 0 to 10168440
Data columns (total 16 columns):
 #   Column                                            Dtype         
---  ------                                            -----         
 0   date                                              datetime64[ns]
 1   precipitation_amount_mm                           float64       
 2   relative_humidity_%                               float64       
 3   specific_humidity_kg/kg                           float64       
 4   surface_downwelling_shortwave_flux_in_air_W m-2   float64       
 5   wind_from_direction_Degrees Clockwise from north  float64       
 6   wind_speed_m/s                                    float64       
 7   max_air_temperature_K                             float64       
 8   min_air_temperature_K                             float64       
 9   burning_index_g_Unitless                          float64       
 10  dead_fuel_moisture_100hr_Percent 

In [5]:
df.describe()

,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa
count,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06,6.501450e+06
mean,8.502061e-01,3.011313e+01,6.529446e-03,2.386129e+02,2.264572e+02,2.979890e+00,2.976179e+02,2.834824e+02,3.709669e+01,1.203044e+01,1.276973e+01,5.019266e+01,4.326740e+00,1.355720e+00
std,4.372622e+00,1.777520e+01,2.913681e-03,8.001784e+01,7.745626e+01,1.259263e+00,7.789443e+00,6.112466e+00,1.911510e+01,4.546625e+00,4.040355e+00,2.097613e+01,1.935780e+00,9.774859e-01
min,0.000000e+00,1.000000e-01,2.400000e-04,4.500000e+00,0.000000e+00,4.000000e-01,2.686000e+02,2.575000e+02,0.000000e+00,1.900000e+00,3.400000e+00,0.000000e+00,2.000000e-01,0.000000e+00
25%,0.000000e+00,1.540000e+01,4.330000e-03,1.651000e+02,1.960000e+02,2.200000e+00,2.921000e+02,2.791000e+02,2.700000e+01,8.200000e+00,9.700000e+00,3.400000e+01,2.700000e+00,6.300000e-01
50%,0.000000e+00,2.680000e+01,6.220000e-03,2.461000e+02,2.410000e+02,2.700000e+00,2.970000e+02,2.834000e+02,3.700000e+01,1.210000e+01,1.290000e+01,4.700000e+01,4.200000e+00,1.090000e+00
75%,0.000000e+00,4.270000e+01,8.460000e-03,3.129000e+02,2.770000e+02,3.500000e+00,3.031000e+02,2.880000e+02,5.000000e+01,1.540000e+01,1.550000e+01,6.600000e+01,5.700000e+00,1.830000e+00
max,2.871000e+02,1.000000e+02,2.392000e-02,3.777000e+02,3.600000e+02,1.620000e+01,3.227000e+02,3.074000e+02,1.510000e+02,3.060000e+01,3.080000e+01,1.100000e+02,1.320000e+01,7.170000e+00


In [6]:
df.head()

,date,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa,geometry
0,2000-01-01,0.0,40.5,0.00600,139.7,222.0,2.1,292.0,282.2,31.0,12.3,12.1,48.0,1.8,0.69,"POLYGON ((-117.97500 33.56667, -117.93333 33.5..."
1,2000-01-01,0.0,40.9,0.00593,136.6,222.0,2.1,291.9,282.2,32.0,12.2,12.0,48.0,1.8,0.70,"POLYGON ((-117.93333 33.56667, -117.89167 33.5..."
2,2000-01-01,0.0,41.2,0.00580,133.2,222.0,2.3,292.1,282.1,33.0,11.8,11.5,50.0,1.9,0.72,"POLYGON ((-117.89167 33.56667, -117.85000 33.5..."
3,2000-01-01,0.0,40.8,0.00577,129.8,218.0,2.4,292.0,281.4,34.0,11.8,11.4,50.0,1.9,0.69,"POLYGON ((-117.85000 33.56667, -117.80833 33.5..."
4,2000-01-01,0.0,38.2,0.00547,126.5,218.0,2.5,292.6,281.0,37.0,10.7,10.5,55.0,2.1,0.78,"POLYGON ((-117.80833 33.56667, -117.76667 33.5..."


In [7]:
perimeters = gpd.read_file('../../data/geoMAC/sd_fire_final_dedupe.shp')
perimeters['date'] = pd.to_datetime(perimeters.final_date, format="%Y-%m-%d")
perimeters.drop(perimeters.columns.difference(['date', 'geometry', 'acres', 'fire_name']), axis=1, inplace=True)
perimeters.crs = {'init' :'epsg:3857'}
perimeters = perimeters.to_crs(epsg=4326)
print(perimeters.shape)
perimeters.head()

C:\Users\Mike\AppData\Local\Continuum\anaconda3\envs\geo\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


(176, 4)


,fire_name,acres,geometry,date
0,AMMO,21493.908203,"POLYGON ((-117.51258 33.39133, -117.51137 33.3...",2007-10-23
1,ANTONIO,1480.468628,"POLYGON ((-117.59738 33.61634, -117.59762 33.6...",2002-05-13
2,Ammo,19834.116000,"POLYGON ((-117.52549 33.35403, -117.52669 33.3...",2007-10-25
3,Ammo,20999.923000,"POLYGON ((-117.43597 33.27178, -117.43773 33.2...",2007-10-26
4,Ammo,21669.283357,"POLYGON ((-117.51469 33.39194, -117.51258 33.3...",2007-10-23


In [8]:
%%time
fire_occurred = gpd.sjoin(df[['date', 'geometry']], perimeters)
fire_occurred = fire_occurred[fire_occurred.date_left == fire_occurred.date_right]
fire_occurred

C:\Users\Mike\AppData\Local\Continuum\anaconda3\envs\geo\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != {'init': 'epsg:4326', 'no_defs': True})
  "(%s != %s)" % (left_df.crs, right_df.crs)


Wall time: 24min 54s


,date_left,geometry,index_right,fire_name,acres,date_right
1201304,2002-05-13,"POLYGON ((-117.64167 33.56667, -117.60000 33.5...",1,ANTONIO,1480.468628,2002-05-13
1201305,2002-05-13,"POLYGON ((-117.60000 33.56667, -117.55833 33.5...",1,ANTONIO,1480.468628,2002-05-13
1940470,2003-10-26,"POLYGON ((-117.05833 33.56667, -117.01667 33.5...",111,MOUNTAIN,10331.393555,2003-10-26
1940471,2003-10-26,"POLYGON ((-117.01667 33.56667, -116.97500 33.5...",111,MOUNTAIN,10331.393555,2003-10-26
1940472,2003-10-26,"POLYGON ((-116.97500 33.56667, -116.93333 33.5...",111,MOUNTAIN,10331.393555,2003-10-26
...,...,...,...,...,...,...
2929950,2005-10-05,"POLYGON ((-116.72500 32.56667, -116.68333 32.5...",8,BORDER#50,3862.903076,2005-10-05
2929951,2005-10-05,"POLYGON ((-116.68333 32.56667, -116.64167 32.5...",8,BORDER#50,3862.903076,2005-10-05
2929997,2005-10-05,"POLYGON ((-116.76667 32.52500, -116.72500 32.5...",8,BORDER#50,3862.903076,2005-10-05
2929998,2005-10-05,"POLYGON ((-116.72500 32.52500, -116.68333 32.5...",8,BORDER#50,3862.903076,2005-10-05


In [9]:
fire_occurred.fire_name.value_counts()

Witch                      557
Harris                     209
Poomacha                   182
CEDAR                       96
Border 3                    88
                          ... 
COMBAT-BASILONE COMPLEX      2
Old                          2
PLEASURE                     2
ANTONIO                      2
GENERAL                      2
Name: fire_name, Length: 71, dtype: int64

In [10]:
df['fire_occurred'] = False
df.loc[fire_occurred.index, 'fire_occurred'] = True
df.loc[fire_occurred.index, 'acres_burned'] = fire_occurred.acres
df.loc[fire_occurred.index, 'fire_name'] = fire_occurred.fire_name
df.head()

,date,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa,geometry,fire_occurred,acres_burned,fire_name
0,2000-01-01,0.0,40.5,0.00600,139.7,222.0,2.1,292.0,282.2,31.0,12.3,12.1,48.0,1.8,0.69,"POLYGON ((-117.97500 33.56667, -117.93333 33.5...",False,NaN,NaN
1,2000-01-01,0.0,40.9,0.00593,136.6,222.0,2.1,291.9,282.2,32.0,12.2,12.0,48.0,1.8,0.70,"POLYGON ((-117.93333 33.56667, -117.89167 33.5...",False,NaN,NaN
2,2000-01-01,0.0,41.2,0.00580,133.2,222.0,2.3,292.1,282.1,33.0,11.8,11.5,50.0,1.9,0.72,"POLYGON ((-117.89167 33.56667, -117.85000 33.5...",False,NaN,NaN
3,2000-01-01,0.0,40.8,0.00577,129.8,218.0,2.4,292.0,281.4,34.0,11.8,11.4,50.0,1.9,0.69,"POLYGON ((-117.85000 33.56667, -117.80833 33.5...",False,NaN,NaN
4,2000-01-01,0.0,38.2,0.00547,126.5,218.0,2.5,292.6,281.0,37.0,10.7,10.5,55.0,2.1,0.78,"POLYGON ((-117.80833 33.56667, -117.76667 33.5...",False,NaN,NaN


In [11]:
df.head()

,date,precipitation_amount_mm,relative_humidity_%,specific_humidity_kg/kg,surface_downwelling_shortwave_flux_in_air_W m-2,wind_from_direction_Degrees Clockwise from north,wind_speed_m/s,max_air_temperature_K,min_air_temperature_K,burning_index_g_Unitless,dead_fuel_moisture_100hr_Percent,dead_fuel_moisture_1000hr_Percent,energy_release_component-g_Unitless,potential_evapotranspiration_mm,mean_vapor_pressure_deficit_kPa,geometry,fire_occurred,acres_burned,fire_name
0,2000-01-01,0.0,40.5,0.00600,139.7,222.0,2.1,292.0,282.2,31.0,12.3,12.1,48.0,1.8,0.69,"POLYGON ((-117.97500 33.56667, -117.93333 33.5...",False,NaN,NaN
1,2000-01-01,0.0,40.9,0.00593,136.6,222.0,2.1,291.9,282.2,32.0,12.2,12.0,48.0,1.8,0.70,"POLYGON ((-117.93333 33.56667, -117.89167 33.5...",False,NaN,NaN
2,2000-01-01,0.0,41.2,0.00580,133.2,222.0,2.3,292.1,282.1,33.0,11.8,11.5,50.0,1.9,0.72,"POLYGON ((-117.89167 33.56667, -117.85000 33.5...",False,NaN,NaN
3,2000-01-01,0.0,40.8,0.00577,129.8,218.0,2.4,292.0,281.4,34.0,11.8,11.4,50.0,1.9,0.69,"POLYGON ((-117.85000 33.56667, -117.80833 33.5...",False,NaN,NaN
4,2000-01-01,0.0,38.2,0.00547,126.5,218.0,2.5,292.6,281.0,37.0,10.7,10.5,55.0,2.1,0.78,"POLYGON ((-117.80833 33.56667, -117.76667 33.5...",False,NaN,NaN


In [12]:
df.fire_occurred.value_counts()

False    6499707
True        1743
Name: fire_occurred, dtype: int64

In [13]:
df.date = df.date.astype(str)
df.to_file("integratedData.shp")